In [52]:
import pandas as pd
import numpy as np
from IPython.display import display
from pathlib import Path
import xlrd

input_list = [
    Path('sokul_1.xls'),
    Path('sokul_2.xls'),
]
result_df = None
for in_path in input_list:
    try:
        xl = pd.ExcelFile(in_path)
    except UnicodeDecodeError:
        xlrd_book = xlrd.open_workbook(in_path, on_demand=True, encoding_override="cp1251")
        xl = pd.ExcelFile(xlrd_book)
    df = xl.parse()
    header_str = "Код"
    cols = list(df.columns)
    header_col = cols[1]
    header_row = df.index.get_loc(df.index[df[header_col] == header_str][0])
    df = xl.parse(header=header_row + 1)
    df = df.rename(columns={
        'Товар': 'Наименование',
        'Шт./упак': 'Упак.',
    })
    df.insert(loc=df.columns.get_loc("Наименование") + 1, column='Производитель', value=pd.Series(np.nan, dtype="string"))
    df.insert(loc=df.columns.get_loc("Цена с НДС") + 1, column='НДС', value=20)
    df['Цена с НДС'] = df['Цена с НДС'].apply(lambda x: x if isinstance(x, float) else np.nan)
    supplier = np.nan
    for index, row in df.iterrows():
        if pd.isna(row['Цена с НДС']) and pd.isna(row['Код УКТ']):
            supplier = row['Наименование']
        else:
            df.at[index, 'Производитель'] = supplier
    df = df[(df['Цена с НДС'].notnull())]
    df = df[['Код', 'Наименование', 'Производитель', 'Цена с НДС', 'НДС', 'Упак.']]
    
    if result_df is None:
        result_df = df
    else:
        result_df = result_df.append(df, ignore_index=True)

display(result_df)

*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'


,Код,Наименование,Производитель,Цена с НДС,НДС,Упак.
0,1020040,Nutrino Каша молочно - рисовая 200г.,каши Nutrino,43.75,20,9
1,1020041,Nutrino Каша молочная - манка с бананом 200г.,каши Nutrino,43.75,20,9
2,1020042,Nutrino Каша молочная - 7 злаков 200г.,каши Nutrino,43.75,20,9
3,1020043,Nutrino Каша безмол. - 5 злаков с яблоком и ба...,каши Nutrino,43.75,20,9
4,1020044,Nutrino Каша молочная - рис и кукуруза с банан...,каши Nutrino,43.75,20,9
...,...,...,...,...,...,...
828,2020950,Avent 150/50 Термоконтейнер на дві пляшки Черв...,Avent,516.23,20,5
829,2020951,Avent 284/03 стерилизатор паровий електр. паро...,Avent,1997.87,20,1
830,2020952,Avent 680/00 промо-пляшечка,Avent,0.01,20,1
831,2020953,Avent 030/07 Пляшечка для годув NATURAL 125мл 1шт,Avent,199.39,20,
